# Поиск критериев платежеспособности клиентов банка <a name="0"></a>

Анализ проведен для кредитного отдела банка. Входные данные от банка — статистика о платёжеспособности клиентов и обезличенные данные по таким параметрам, как количество детей, рабочий стаж, возраст, образование, семейное положение, пол, должность, доход и цель кредитования, а также отметка о возврате кредита в срок.  
  
Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.  

____
### Оглавление

#### [Часть 1. Первичное изучение полученных данных](#1)

   [1.1 Загрузка необходимых для анализа библиотек и файлов](#1.1)  
   [1.2 Общее описание содержания данных](#1.2)  
   [1.3 Вывод: обнаруженные проблемы в данных](#1.3)  

#### [Часть 2. Предобработка данных](#2)

   [2.1 Обработка пропусков](#2.1)  
   [2.2 Замена типов данных](#2.2)  
   [2.3 Обработка дубликатов](#2.3)   
   [2.4 Категоризация кредитных целей](#2.4)    
       [2.4.1 Выводы: основные кредитные цели](#2.4.1)  
   [2.5 Категоризация клиентов банка по персональным данным](#2.5)  
       [2.5.1 Выводы: ключевые категории клиентов](#2.5.1) 

#### [Часть 3. Критерии оценки платежеспособности клиентов банка](#3)

   [3.1 Гипотеза: есть зависимость между наличием детей и возвратом кредита в срок](#3.1)  
   [3.2 Вывод](#3.2)  
   [3.3 Гипотеза: есть зависимость между семейным положением и возвратом кредита в срок](#3.3)  
   [3.4 Вывод](#3.4)    
   [3.5 Гипотеза: есть зависимость между уровнем дохода и возвратом кредита в срок](#3.5)  
   [3.6 Вывод](#3.6)   
   [3.7 Гипотеза: разные цели кредита влияют на его возврат в срок](#3.7)  
   [3.8 Вывод](#3.8)   

#### [Часть 4. Итоговые выводы исследования](#4)
____


## Часть 1. Первичное изучение полученных данных <a name="1"></a>

### 1.1 Загрузка необходимых для анализа библиотек и файлов <a name="1.1"></a>

In [1]:
import pandas as pd
from pymystem3 import Mystem # импортируем для будущей работы с данными pymystem3 и Counter
from collections import Counter
m = Mystem()

data_bank_clients = pd.read_csv("/datasets/1_bank_data.csv")
data_bank_clients.head() 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [2]:
# общая информация о данных таблицы
data_bank_clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
# проверка на наличие дубликатов
data_bank_clients.duplicated().sum()

54

In [4]:
# далее изучаем данные из таблицы по ключевым колонкам
data_bank_clients["children"].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [5]:
data_bank_clients["family_status"].value_counts()
# для проверки данных в "family_status" можно использовать столбец "family_status_id"

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [6]:
data_bank_clients["dob_years"].unique()
# обратить внимание на возраст — 0

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75])

In [7]:
data_bank_clients["education"].unique() 
# для проверки данных в "education" можно использовать столбец "education_id"

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [8]:
# изучение данных в столбце "total_income"
data_bank_clients.sort_values("total_income", ascending=False).head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12412,0,-1477.438114,44,высшее,0,женат / замужем,0,M,компаньон,0,2.265604e+06,ремонт жилью
19606,1,-2577.664662,39,высшее,0,женат / замужем,0,M,компаньон,1,2.200852e+06,строительство недвижимости
9169,1,-5248.554336,35,среднее,1,гражданский брак,1,M,сотрудник,0,1.726276e+06,дополнительное образование
20809,0,-4719.273476,61,среднее,1,Не женат / не замужем,4,F,сотрудник,0,1.715018e+06,покупка жилья для семьи
17178,0,-5734.127087,42,высшее,0,гражданский брак,1,M,компаньон,0,1.711309e+06,сыграть свадьбу
17503,0,-2285.476482,43,среднее,1,женат / замужем,0,M,компаньон,0,1.597613e+06,операции с недвижимостью
18368,1,-333.935516,41,ВЫСШЕЕ,0,гражданский брак,1,M,компаньон,0,1.551153e+06,свадьба
18353,1,-3173.282035,41,высшее,0,Не женат / не замужем,4,F,компаньон,0,1.427934e+06,автомобиль
15268,1,-10207.448165,64,высшее,0,в разводе,3,M,компаньон,0,1.350246e+06,жилье
11071,1,-1851.200013,36,высшее,0,гражданский брак,1,F,сотрудник,0,1.286281e+06,покупка коммерческой недвижимости


### 1.2 Общее описание содержания данных <a name="1.2"></a>

Всего в таблице 12 столбцов, которые содержат следующую информацию и сообветствующие типы данных: 
  
1) *children* — количество детей в семье — *int64* (корректный тип данных)  
2) *days_employed* — трудовой стаж в днях — **float64** (некорректный тип данных, необходима замена на *int64*)  
3) *dob_days* — возраст клиента в годах — *int64* (корректно)  
4) *education* — образование клиента — *object* (корректно)  
5) *education_id* — идентификатор образования — *int64* (корректно, логическая переменная)  
6) *family_status* — семейное положение — *object* (корректно)  
7) *family_status_id* — идентификатор семейного положения — *int64* (корректно, логическая переменная) 
8) *gender* — пол клиента — *object* (корректно)  
9) *income_type* — тип занятости — *object* (корректно)  
10) *debt* — имел ли задолженность по возврату кредитов — *int64* (корректно, логическая переменная)  
11) *total_income* — доход в месяц — **float64** (некорректный, необходима замена на *int64*)  
12) *purpose* — цель получения кредита — *object* (корректно)   
  
Количество значений в столбцах различается. Это говорит о том, что **в данных есть пропущенные значения значения** в столбцах "days_employed" и "total_income", здесь же наблюдаются проблемы с качеством данных:  
* в столбце "days_employed" есть отрицательные значения и выбросы, данные ошибки могли появится при переводе рабочего стажа в дни из других величин, чтобы решить данную проблему необходимо обратиться к коллеге, который прислал исходный файл,  
* в столбце "total_income" часть данных записана в виде экспоненциального числа, возможны выбросы, оценить которые можно только после перевода данных в целые числа.  
  
**Количество дубликатов:** 54

### 1.3 Вывод: обнаруженные проблемы в данных <a name="1.3"></a>

Каждая строка таблицы содержит информацию о клиентах банка, целях кредитования и наличии у них задолженностей в кредитной истории. Для проверки рабочих гепотез особенно ценны столбцы: children, family_status	и family_status_id, total_income, purpose. Данные о наличии или отсутствии задолженностей находятся в столбце debt.  
  
Для работы с выбросами и ошибками в столбце total_income могут понадобится данные об образовании из столбца education/education_id и типе занятости income_type.  
  
Проблемы с данными, которые нужно решать:  
* ошибки в столбце children, которые могли возникнуть из-за ручного ввода данных (значения -1 и 20, где ошибка может быть в знаке "-" и лишнем "0"),  
* пропуски и некорректный тип данных в столбце total_income, ошибки могли появиться при переводе значений из другой валюты или при из-за некорректного написания (например, с точками/запятыми).  
  
***
Столбец days_employed можно исключить из датафрейма, так как он не влияет на проверку рабочих гипотез и содержит некорректную информацию.

In [9]:
# удалаем столбец days_employed методом .drop()
data_bank_clients.drop(["days_employed"], axis = 1, inplace=True)
data_bank_clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   dob_years         21525 non-null  int64  
 2   education         21525 non-null  object 
 3   education_id      21525 non-null  int64  
 4   family_status     21525 non-null  object 
 5   family_status_id  21525 non-null  int64  
 6   gender            21525 non-null  object 
 7   income_type       21525 non-null  object 
 8   debt              21525 non-null  int64  
 9   total_income      19351 non-null  float64
 10  purpose           21525 non-null  object 
dtypes: float64(1), int64(5), object(5)
memory usage: 1.8+ MB


## Часть 2. Предобработка данных <a name="2"></a>

### 2.1 Обработка пропусков <a name="2.1"></a>

In [10]:
# суммарное количество пропусков
data_bank_clients.isnull().sum()

children               0
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [11]:
# убираем пропуски
data_bank_clients = data_bank_clients.dropna().reset_index(drop=True)
data_bank_clients.isnull().sum()

children            0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Вывод

Пропущенные значения обнаружены в столбце "total_income", где записана информация об уровне дохода клиента — 2174 пропуска из 21525 значений (10%).  
  
Эти пропуски могли образоваться из-за нежелания респондентов предоставлять данные о доходах, при этом графы об образовании типе занятости без пропусков, что позволяет отнести механизм формирования пропусков к MAR (Missing At Random). В этом случае удаление пустых значений приемлемо. 
   
Подтвержденный уровень дохода — один из основных показателей платежеспособности клиента банка, при этом замена пропусков в данном случае может повлиять на статистические результаты, поэтому несмотря на высокий процент пропусков 10%, было принято решение удалить строки с пропущенными значениями.

### 2.2 Замена типов данных <a name="2.2"></a>

In [12]:
# заменяем тип данных в столбце "total_income"
data_bank_clients["total_income"] = data_bank_clients["total_income"].astype("int")

In [13]:
# проверка результатов замены
data_bank_clients.sort_values("total_income", ascending=False).head()

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
11189,0,44,высшее,0,женат / замужем,0,M,компаньон,0,2265604,ремонт жилью
17638,1,39,высшее,0,женат / замужем,0,M,компаньон,1,2200852,строительство недвижимости
8291,1,35,среднее,1,гражданский брак,1,M,сотрудник,0,1726276,дополнительное образование
18700,0,61,среднее,1,Не женат / не замужем,4,F,сотрудник,0,1715018,покупка жилья для семьи
15461,0,42,высшее,0,гражданский брак,1,M,компаньон,0,1711309,сыграть свадьбу


In [14]:
data_bank_clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19351 entries, 0 to 19350
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          19351 non-null  int64 
 1   dob_years         19351 non-null  int64 
 2   education         19351 non-null  object
 3   education_id      19351 non-null  int64 
 4   family_status     19351 non-null  object
 5   family_status_id  19351 non-null  int64 
 6   gender            19351 non-null  object
 7   income_type       19351 non-null  object
 8   debt              19351 non-null  int64 
 9   total_income      19351 non-null  int64 
 10  purpose           19351 non-null  object
dtypes: int64(6), object(5)
memory usage: 1.6+ MB


### Вывод

При первом знакомстве с данными обнаружены проблемы с типом и отображением данных в столбце "total_income" (доход в месяц). С помощью метода `astype()` был заменен тип данных "float64" на "int64". Это помогло решить проблему с записью данных и заменить экпоненциальную запись на целочисленную.   

### 2.3 Обработка дубликатов <a name="2.3"></a>

In [15]:
data_bank_clients = data_bank_clients.drop_duplicates().reset_index(drop=True)
data_bank_clients.duplicated().sum()

0

In [16]:
# уникальные категории "education"
data_bank_clients["education"].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [17]:
# количество скрытых дубликатов
data_bank_clients["education"] = data_bank_clients["education"].str.lower()
data_bank_clients.duplicated().sum()

0

### Вывод

В таблице на этапе изучения DataFrame выявлены дубликаты методом `duplicated()` и подсчитано их количество с помощью функции `sum()` — 54 строки. Дубликаты могли появиться в результате сбоя в записи данных.
  
Дубликаты были удалены с помощью метода `drop_duplicates()`, чтобы переиндексировать строки после изменения DataFrame был использован метод `reset_index()` с параметром `drop=True` (удалены старые индексы). "Скрытые" дубликаты не обнаружены.

### 2.4 Категоризация кредитных целей <a name="2.4"></a>

In [18]:
# применим метод лемматизации для выявления категорий
data_bank_clients["lemmas_purpose"] = data_bank_clients["purpose"].apply(m.lemmatize)
Counter(data_bank_clients["lemmas_purpose"].sum())

Counter({'покупка': 5353,
         ' ': 30248,
         'жилье': 4035,
         '\n': 19351,
         'приобретение': 419,
         'автомобиль': 3897,
         'дополнительный': 810,
         'образование': 3597,
         'сыграть': 693,
         'свадьба': 2099,
         'операция': 2334,
         'с': 2619,
         'на': 2016,
         'проведение': 685,
         'для': 1158,
         'семья': 570,
         'недвижимость': 5723,
         'коммерческий': 1178,
         'жилой': 1126,
         'строительство': 1701,
         'собственный': 560,
         'подержать': 437,
         'свой': 2013,
         'со': 559,
         'заниматься': 797,
         'получение': 1179,
         'сделка': 844,
         'подержанный': 435,
         'высокий': 1243,
         'профильный': 389,
         'сдача': 588,
         'ремонт': 542})

In [19]:
def lemma_goal(lemma_list):
    if "автомобиль" in lemma_list:
        return "автомобиль"
    if "образование" in lemma_list:
        return "образование"
    if "свадьба" in lemma_list:
        return "свадьба"
    if "ремонт" in lemma_list:
        return "ремонт"
    if ("недвижимость" in lemma_list) or ("жилье" in lemma_list):
        return "недвижимость"
    return "пусто"

In [20]:
# проверим результат
data_bank_clients["credit_goal"] = data_bank_clients["lemmas_purpose"].apply(lemma_goal)
Counter(data_bank_clients["credit_goal"])

Counter({'недвижимость': 9216,
         'автомобиль': 3897,
         'образование': 3597,
         'свадьба': 2099,
         'ремонт': 542})

In [21]:
# убедимся, что "ремонт" можно выделить в отдельную категорию 
data_bank_clients[data_bank_clients["credit_goal"] == "ремонт"]["purpose"].unique()

array(['ремонт жилью'], dtype=object)

### 2.4.1 Выводы <a name="2.4.1"></a>

Для проверки гипотезы необходимо было категоризировать цели кредитования по колонке "purpose". С помощью лемматизации `m.lemmataze` и `Counter` выявлены уникально встречающиеся слова и выделены первичные цели кредитования: жилье, автомобиль, образование, свадьба, недвижимость, строительство и ремонт. Из полученных категорий "жилье" и "недвижимость" были объединены в единую категорию "недвижимость", так как схожи по запросу, в эту категорию вошла покупка частной и коммерческой недвижимости, а так же строительство.  
  
Две категории "ремонт" и "строительство" на первый взгляд могли быть объединены, но при проверке выяснилось, что под строительством чаще имеется в виду "недвижимость", а также предположительно эти категории значительно отличаются по сумме кредитования. Поэтому было принято решение выделить "ремонт" в отдельную категорию, а "строительство" включить в "недвижимость".  
  
Итого выделены следующие ключевые леммы-цели кредитования и добавлены в таблицу: недвижимость, автомобиль, образование, свадьба, ремонт.     

### 2.5 Категоризация клиентов банка по персональным данным <a name="2.5"></a>

In [22]:
# подготовка данных в столбце children для категоризации
data_bank_clients["children"].value_counts()

 0     12710
 1      4343
 2      1851
 3       294
 20       67
-1        44
 4        34
 5         8
Name: children, dtype: int64

Вероятнее всего в значениях -1 и 20 появились лишние знаки при ручном заполнении поля. Заменим данные на 1 и 2 соответственно, это существенно не повлияет на категорию "есть дети" — для дальнейшего анализа их количество не будет играть ключевой роли.

In [23]:
data_bank_clients.loc[data_bank_clients["children"] == -1, "children"] = 1
data_bank_clients.loc[data_bank_clients["children"] == 20, "children"] = 2
data_bank_clients["children"].value_counts()

0    12710
1     4387
2     1918
3      294
4       34
5        8
Name: children, dtype: int64

In [24]:
# категоризируем данные для проверки первой гипотезы:
# есть ли зависимость между наличием детей и возвратом кредита в срок
def child_status(row):
    if row == 0:
        return "нет детей"
    return "есть дети"

# сохраним категории в новый столбец
data_bank_clients["child_status"] = data_bank_clients["children"].apply(child_status)
data_bank_clients.head()

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas_purpose,credit_goal,child_status
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",недвижимость,есть дети
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль,есть дети
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",недвижимость,нет детей
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",образование,есть дети
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба,нет детей


In [25]:
# изучим данные для категоризации по столбцу о семейном статусе
data_bank_clients.groupby(["family_status", "family_status_id"]).agg({"family_status" : "count"})

,,family_status
family_status,family_status_id,
Не женат / не замужем,4,2525
в разводе,3,1083
вдовец / вдова,2,865
гражданский брак,1,3735
женат / замужем,0,11143


In [26]:
# выделим категории
def family_debt_status(row):
    if row == 0:
        return "в браке"
    return "не в браке"

# сохраним категории в новый столбец
data_bank_clients["family_debt_status"] = data_bank_clients["family_status_id"].apply(family_debt_status)
data_bank_clients.head()

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas_purpose,credit_goal,child_status,family_debt_status
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",недвижимость,есть дети,в браке
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль,есть дети,в браке
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",недвижимость,нет детей,в браке
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",образование,есть дети,в браке
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба,нет детей,не в браке


In [27]:
# категоризируем данные по уровню дохода
income_category = data_bank_clients["total_income"].describe()
income_category.astype("int")

count      19351
mean      167421
std       102971
min        20667
25%       103053
50%       145017
75%       203434
max      2265604
Name: total_income, dtype: int64

In [28]:
# выделяем 4 группы: меньше 25%, 25%-50%, 50%-75% и больше 75%
def income_status(row):
    if row <= 103053:
        return "ниже среднего"
    if row <= 145017:
        return "средний"
    if row <= 203434:
        return "высокий"
    return "богатство"
# сохраним категории в новый столбец
data_bank_clients["income_status"] = data_bank_clients["total_income"].apply(income_status)
data_bank_clients.head()

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas_purpose,credit_goal,child_status,family_debt_status,income_status
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",недвижимость,есть дети,в браке,богатство
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль,есть дети,в браке,средний
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",недвижимость,нет детей,в браке,высокий
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",образование,есть дети,в браке,богатство
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба,нет детей,не в браке,высокий


In [29]:
# выделим два словаря для работы с данными об образовании и семейном статусе
family_status_dict = data_bank_clients[["family_status", "family_status_id"]]
family_status_dict.drop_duplicates().reset_index(drop=True)

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


In [30]:
education_dict = data_bank_clients[["education", 'education_id']]
education_dict.drop_duplicates().reset_index(drop=True)

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


In [31]:
# удалим лишние столбцы в датафрейме
data_bank_clients.drop(["education", "family_status", "purpose", "lemmas_purpose"], axis = 1, inplace=True)
data_bank_clients.head()

,children,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,credit_goal,child_status,family_debt_status,income_status
0,1,42,0,0,F,сотрудник,0,253875,недвижимость,есть дети,в браке,богатство
1,1,36,1,0,F,сотрудник,0,112080,автомобиль,есть дети,в браке,средний
2,0,33,1,0,M,сотрудник,0,145885,недвижимость,нет детей,в браке,высокий
3,3,32,1,0,M,сотрудник,0,267628,образование,есть дети,в браке,богатство
4,0,53,1,1,F,пенсионер,0,158616,свадьба,нет детей,не в браке,высокий


### 2.5.1 Выводы <a name="2.5.1"></a>

Для категоризации данных написаны функции и применены к датафрейму с помощью метода `apply()`. Проведена категоризация по наличию детей, семейному положению и уровню дохода. Категории по целям кредитования выделены ранее с помощью лемматизации.   
  
**Пять категорий по целям кредитования:** недвижимость, автомобиль, образование, свадьба, ремонт.
  
**Две категории по наличию детей:** есть дети и нет детей. Некорректные значения "-1" заменены на "1", а "20" на "2" — предположительно небольшой процент ошибок (0,67% от общего количества записей) был связан с ручным вводом данных. Эти значения включены в категорию "есть дети".  
  
**Две категории по семейному положению:** в браке и не в браке. Изначально предполагалось выделить третью категорию "гражданский брак", однако при дальнейшей работе с данными было выявлено, что показатели по наличию задолженностей по кредиту в категориях "гражданский брак" и "не в браке" различается незначительно и может быть объединен. Гипотеза, что "гражданский брак" позволяет сделать вывод о наличии финансовой поддержки по кредиту (как в браке), не нашла подтверждения. В связи с этим для удобства работы с данными оставлены только две категории.

**Четыре категории по уровню дохода:** ниже среднего, средний, высокий и богатство. Чтобы выявить категории по уровню доходов был применен метод `describe()` и выявлены соответственно 4 группы: меньше 25%, 25%-50%, 50%-75% и больше 75%.

## Часть 3. Критерии оценки платежеспособности клиентов банка <a name="3"></a>

### 3.1 Гипотеза: есть зависимость между наличием детей и возвратом кредита в срок <a name="3.1"></a>

In [32]:
# сводная таблица
children_pivot = data_bank_clients.pivot_table(index=["child_status"], 
                                              columns=["debt"], values="children", 
                                              aggfunc="count")
children_pivot.reset_index()

debt,child_status,0,1
0,есть дети,6022,619
1,нет детей,11758,952


In [33]:
# процент клиентов, имеющих задолженность
children_pivot["debt_yes_percent"] = (children_pivot[1] * 100) / (children_pivot[0] + children_pivot[1])
children_pivot.reset_index()

debt,child_status,0,1,debt_yes_percent
0,есть дети,6022,619,9.320885
1,нет детей,11758,952,7.490165


In [34]:
# дополнительное исследование: зависимость задолженности от наличия детей и семейного статуса
children_family_pivot = data_bank_clients.pivot_table(index=["child_status", "family_debt_status"], 
                                              columns=["debt"], values="children", 
                                              aggfunc="count")
children_family_pivot.reset_index()

debt,child_status,family_debt_status,0,1
0,есть дети,в браке,4017,385
1,есть дети,не в браке,2005,234
2,нет детей,в браке,6280,461
3,нет детей,не в браке,5478,491


In [35]:
children_family_pivot["debt_yes_percent"] = (children_family_pivot[1] * 100) / (children_family_pivot[0] + children_family_pivot[1])
children_family_pivot.reset_index()

debt,child_status,family_debt_status,0,1,debt_yes_percent
0,есть дети,в браке,4017,385,8.746025
1,есть дети,не в браке,2005,234,10.451094
2,нет детей,в браке,6280,461,6.838748
3,нет детей,не в браке,5478,491,8.225833


### 3.2 Вывод <a name="3.2"></a>

Из всех клиентов с детьми задолженность имели: 9,32%  
Из всех клиентов без детей задолженность имели: 7,49%  
  
Наличие детей может влиять на увеличение риска образования задоленности по кредиту, однако, рассматривать этот фактор стоит в совокупности с другими, например, при рассмотрении зависимости наличия задолженности по кредиту от наличия детей и семейного статуса показало, что наибольший риск в неполных семьях (не в браке) с детьми — 10,45%, а наименьший риск в семьях без детей, состоящих в браке — 6,84%.  

### 3.3 Гипотеза: есть зависимость между семейным положением и возвратом кредита в срок <a name="3.3"></a>

In [36]:
family_pivot = data_bank_clients.pivot_table(index=["family_debt_status"], 
                                              columns=["debt"], values="children", 
                                              aggfunc="count")
family_pivot.reset_index()

debt,family_debt_status,0,1
0,в браке,10297,846
1,не в браке,7483,725


In [37]:
family_pivot["debt_yes_percent"] = (family_pivot[1] * 100) / (family_pivot[0] + family_pivot[1])
family_pivot.reset_index()

debt,family_debt_status,0,1,debt_yes_percent
0,в браке,10297,846,7.592210
1,не в браке,7483,725,8.832846


### 3.4 Вывод <a name="3.4"></a>

Процент задолженности среди клиентов, не состоящих в браке — 8,83% , выше, чем среди клиентов, состоящих в браке — 7,59%. Разница составляет 1,24%. Наличие официально оформленных семейных отношений может считаться положительным фактором для одобрения кредита. При этом, как мы выяснили выше, этот фактор может обрести большее влияние при комплексном рассмотрении семейного положения и наличия детей.

### 3.5 Гипотеза: есть зависимость между уровнем дохода и возвратом кредита в срок <a name="3.5">

In [38]:
income_pivot = data_bank_clients.pivot_table(index=["income_status"], 
                                              columns=["debt"], values="children", 
                                              aggfunc="count")
income_pivot.reset_index()

debt,income_status,0,1
0,богатство,4497,341
1,высокий,4411,426
2,ниже среднего,4455,383
3,средний,4417,421


In [39]:
income_pivot["debt_yes_percent"] = (income_pivot[1] * 100) / (income_pivot[0] + income_pivot[1])
income_pivot.reset_index()

debt,income_status,0,1,debt_yes_percent
0,богатство,4497,341,7.048367
1,высокий,4411,426,8.807112
2,ниже среднего,4455,383,7.916494
3,средний,4417,421,8.701943


### 3.6 Вывод <a name="3.6"></a>

Уровень дохода не имеет значительного влияния: вне зависимости от уровня дохода процент задолженности в категориях "ниже среднего", "средний" и "высокий" колеблится от 7,9% до 8,8%, только в случае "богатство" процент порядком ниже — 7%.

### 3.7 Гипотеза: разные цели кредита влияют на его возврат в срок <a name="3.7">

In [40]:
credit_goal_pivot = data_bank_clients.pivot_table(index=["credit_goal"], 
                                              columns=["debt"], values="children", 
                                              aggfunc="count")
credit_goal_pivot.reset_index()

debt,credit_goal,0,1
0,автомобиль,3530,367
1,недвижимость,8532,684
2,образование,3266,331
3,ремонт,511,31
4,свадьба,1941,158


In [41]:
credit_goal_pivot["debt_yes_percent"] = (credit_goal_pivot[1] * 100) / (credit_goal_pivot[0] + credit_goal_pivot[1])
credit_goal_pivot.reset_index()

debt,credit_goal,0,1,debt_yes_percent
0,автомобиль,3530,367,9.417501
1,недвижимость,8532,684,7.421875
2,образование,3266,331,9.202113
3,ремонт,511,31,5.719557
4,свадьба,1941,158,7.527394


### 3.8 Вывод <a name="3.8"></a>

Выявлена зависимость наличия задолженности от цели кредитования:  
— наиболее высокий риск для целей "автомобиль" и "образование" ~9,3%  
— средний показатель у целей "недвижимость" и "свадьба" ~7,45%  
— наиболее низкий риск у цели "ремонт" — 5,7%  

## Часть 4. Итоговые выводы исследования <a name="4"></a>

Наличие детей в семье увеличивает вероятность образования задолженности, однако, рекомендуется рассматривать эту метрику в совокупности с семейным статусом.   Процент задолженности:  
* клиенты с детьми, не в браке — 10,45%,   
* клиенты без детей, в браке — 6,84%,  
* клиенты с детьми, в браке — 8,75%,  
* клиенты без детей, не в браке — 8,23%. 
  
Наличие статуса "женат/замужем" может положительно влиять на кредитную привлекательность клиента для банка, однако, не может иметь наибольший вес при оценке платежеспособности. Это фактор стоит учитывать в совокупности с наличием в семье детей, чтобы получить более точные оценки на этапе кредитного скоринга.
  
Задолженность не зависит от уровня дохода. В данном случае более показательными будут другие факторы: наличие детей, семейное положение и другие.  
   
Есть зависимость между целью кредита и риском образования задолженности — их необходимо учитывать при построении модели кредитного скоринга.

____
**<center>[Перейти в начало исследования](#0)</center>**